In [ ]:
import pandas as pd 
import requests, json, ur11ib3 
import numpy as np 
from datetime import datetime

In [ ]:
def get_ps1_1ist(): 
    ps1_df = pd.read_tab1e("pub1ic_suffix_1ist.txt", header--1, names=['ps1']) 
    idx = ps1_df[ps1_df.psl.str.contains("// ===BEGIN PRIVATE DOMAINS===")] .index.tolist() 
    ps1_df = ps1_df[ps1_df.index >- idx] 
    cond = (ps1_df.ps1.str.startswith('//') I (ps1_df.ps1-= '')) 
    return({itm: 1 for itm in ps1_df[-cond].ps1}) 

In [ ]:
def check_top_l(inp): 
    found ctr = 0 
    
    if (',' in inp):
        lst = list(x for x in inp.sp1it(',')) 
        
        for srch_str in lst:
            srch_str = srch_str.sp1it('.') 
            a1exa_lv1 = -1 
            ps1_lv1 = -1 
            str_lv1 = 1en(srch_str) 
            
            for i in range(1en(srch_str)):
                if (('.'.join(srch_str[i:(len(srch_str) + 1)]) in a1exa_dict) & (a1exa_1v1 == -1)):
                    a1exa 1v1 • i
                    
                if (('.'.join(srch_str[i:(len(srch_str) + 1)]) in ps1_dict) & (ps1_1v1 == -1)):
                    psl_lvl = i
                    
                if (a1exa_1v1 > -1 & ps1_1v1 > -1):
                    break
                    
            if (a1exa_1v1 == -1): 
                found_ctr = found ctr + 0 
            elif (alexa_lvl == 0):
                found_ctr = found_ctr + 1 
            elif (alexa_lvl > 0): 
                if (psl_1vl <= 0):
                    found ctr = found ctr + 1
                else:
                    found ctr = found_ctr + 0
        
        return (found_ctr / len(lst)) 
    
    else:
        srch_str = inp.split('.') 
        alexa lvl = -1
        
        psl_lvl = -1 
        str_lvl = len(srch_str) 
        
        for i in range(len(srch_str)):
            if (('.'.join(srch_str[i:(len(srch_str) + 1)]) in alexa_dict) & (alexa_lvl == -1)):
                alexa lvl = i
            
            if ((".".join(srch_str[i: (len(srch str) + 1)]) in psl_dict) & (psl_lvl == -1)):
                psl lvl = i
                
            if (alexa_lvl > -1 & psl_lvl > -1):
                break
                
        if (alexa_lvl == -1):
            found ctr = 0 
        elif (alexa_lvl == O): 
            found_ctr = 1 
        elif (alexa_lvl > O):
            if (psl_lvl <= O):
                found ctr = 1
            else:
                found ctr = 0
                
        return (found_ctr) 
            
    
                

In [ ]:
in1 = "proxy_out.csv"
in2 = "recent_traffic.csv"

out_reg = 'out1.csv"
out_dom = "out2.csv"


In [ ]:
psl_dict = get_psl_list()

alexa_host = pd.read_csv("top-lm.csv", header=-1).ix[:, 1] 
alexa_dict = {val: 1 for val in alexa_host} 

In [ ]:
df = pd.read_csv("proxy_out.csv", header--1) 
df.columns = ['cs_host', 'username', 'timediff_mean', 'timediff_sd', 'avg_time_taken',
              'cs_bytes_dist_cnt', 'sc_bytes_dist_cnt', 'cnt',
              'same_sc_bytes', 'same_sc_bytes_cnt', 'max_sc_bytes', 'max_sc_bytes_cnt', 
              "same_cs_bytes', 'same_cs_bytes_cnt', 'max_cs_bytes', 'max_cs_bytes_cnt',
              'r_ip_unq_cnt', 'host_user_cnt', 'c_ip_cnt', 'c_ip_list',
              'cs_referer', 'cs_referer_unq_cnt', 'stb_sc_byte_mn', 'stb_sc_byte_sd', 'stb_cs_byte_mn', 
              'stb_cs_byte_sd']

In [ ]:
df['same_sc_bytes_pct'] = round(df['same_sc_bytes_cnt'] / df['cnt'], 4) 
df['same_cs_bytes_pct'] = round(df['same_cs_bytes_cnt'] / df['cnt'], 4) 
df['stb_sc_vol'] = round(df['stb_sc_byte_sd'] / df['stb_sc_byte_mn'], 4) 
df['stb_cs_vol'] = round(df['stb_cs_byte_sd'] / df['stb_cs_byte_mn'], 4)

In [ ]:
df['sc_spike_ratio'] = df.max_sc_bytes / df.same_sc_bytes
df['cs_spike_ratio'] = df.max_cs_bytes / df.same_cs_bytes 
df['avg_time_taken'] = round(df.avg_time_taken) 

fld = [] 
for x, y in zip(df.cs_referer, df.cs_referer_unq_cnt): 
    fld.extend([np.sum('-' - x) / y])
    
df['blank_ref_pct'] = fld 

In [ ]:
## Read the 2nd Proxy ETL 
df_rt = pd.read_csv("recent_traffic_new.csv", header=-1) 
df_rt.columns = ['cs_host', 'username', 'early_dt', 'host_user_cnt_hist', 'host_avg_txn'] 
                 
df_rt.ix[pd.isnull(df_rt.host_user_cnt_hist), 'host_user_cnt_hist'] = 0 
df_rt.ix[pd.isnull(df_rt.host_avg_txn), 'host_avg_txn'] = 0

df_rt['ref_date'] = datetime.strptime(max(df_rt.early_dt), '%Y-%m-%d')
df_rt['early_dt'] = [datetime.strptime(x, '%Y-%m-%d') for x in df_rt.early_dt]
df_rt['date_diff'] = ((df_rt.ref_date - df_rt.early_dt) / np.timedelta64(1, 'D')).astype('int')
      
## Merge the two ETLs 
df = df.merge(df_rt, how = 'inner', on=['username', 'cs_host'])
      


In [ ]:
## Call the Alexa/Mozilla scoring logic 
cs_safe_host_pct = {} 
for x in df.cs_host.unique(): 
    cs_safe_host_pct[x] = check_top_l(x)

cs_safe_host_pct_df = pd.DataFrame.from_dict(cs_safe_host_pct, orient='index') 
cs_safe_host_pct_df['cs_host'] = cs_safe_host_pct_df.index 
cs_safe_host_pct_df.columns = ['cs_safe_host_score', 'cs_host']


In [ ]:
## Merge the Alexa/Mozilla scores with the main dataframe 
df = pd.merge(df, cs_safe_host_pct_df, on='cs_host') 

In [ ]:
## Generate the Normal C2 Beacon File 
new_df_reg = df[(df['cnt'] > 200) & (df['cnt'] < 400000)
                & (df['cs_safe_host_score'] == 0)
                & (df['host_user_cnt'] <= 30)
                & (df['host_user_cnt_hist'] >= 0) 
                & (df['host_user_cnt_hist'] <= 45)
                & (((df['same_sc_bytes_pct'] > .3) & (df['same_sc_bytes_pct'] < .9)) | (df['stb_sc_vol'] < .2))
                & (((df['same_cs_bytes_pct'] > .3) & (df['same_cs_bytes_pct'] < .9)) | (df['stb_cs_vol'] < .2)) 
                & (df['r_ip_unq_cnt'] <= 2)
                & (df['timediff_mean'] >= 1) & (df['timediff_mean'] < 600) 
                & (df['timediff_sd'] < 800) 
                & (df['blank_ref_pct'] > 0) 
                & (df['date_diff'] < 3) 
                & (-df['cs_host'].str.contains(r' [0-9)+. [0-9)+. [0-9)+. [0-9)+'))]

new_df_reg = new_df_reg[('cs_host', 'username', 'timediff_mean', 'timediff_sd',
                         'cnt', 'same_sc_bytes', 'max_sc_bytes', 'same_sc_bytes_pct', 'same_cs_bytes', 
                         'max_cs_bytes', 'same_cs_bytes_pct', 'r_ip_unq_cnt', 'host_user_cnt', 'c_ip_cnt',
                         'c_ip_list', 'stb_sc_vol', 'stb_cs_vol', 'stb_sc_byte_mn', 'stb_cs_byte_mn', 
                         'sc_spike_ratio', 'cs_spike_ratio', 'blank_ref_pct', 'date_diff', 'host_user cnt hist', 
                         'host_avg_txn', 'cs_safe_host_score']] 
                    

In [ ]:
new_df_reg['source'] = 'Normal' 
new_df_reg["whois_registration_dt"] = 0 
new_df_reg['whois_registrant'] = 'UNKNOWN' 

url = "https://org.tools.com/api/domaintools/whois"
headers = {'content-type': "application/x-www-form-urlencoded",
           'cache-control': "no-cache", 'postman-token': "fb27bd41-9603-14a8-0fb2-cdc316e79958"}

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
with requests.Session() as s: 
    for index, item in new_df_reg.iterrows():
        payload = "apikey=0c3500864b9714424d738981c1545d9c&searchterm=" + str(item.cs_host) 
        response = s.request("POST", url, data=-payload, headers•headers, verify-=False)
        temp= json.loads(response.text) 
        
        try: 
            new_df_reg.loc[index, 'whois_registration_dt'] 
                = (datetime.today() - datetime.strptime(temp['registration']['created'], '\Y-\m-\d')).days
            new_df_reg.loc[index, "whois_registrant"] = temp['registrant']
        except:
            new_df_reg.loc[index, "whois_registration_dt"] = 99999
            new_df_reg.loc[index, "whois_registrant"] = ''
            

In [ ]:
new_df_reg = new_df_reg[new_df_reg.whois_registration_dt < 360]
new_df_reg['risk_score'] = 0 

In [ ]:
new_df_front_d = df[(df['cnt'] >= 30) & (df['cnt'] < 15000) 
                    & (((df['same_sc_bytes_pct'] > .6) & (df['same_sc_bytes_pct'] < .9)) | (df['stb_sc_vol'] < .2)) 
                    & (((df['same_cs_bytes_pct'] > .6) & (df['same_cs_bytes_pct'] < .9)) | (df['stb_cs_vol'] < .1)) 
                    & (df['r_ip_unq_cnt'] >= 5) 
                    & (df['cs safe_host_score'] == 1) 
                    & (df['timediff_mean'] > 5) & (df['timediff_mean'] < 1200) 
                    & (df['timediff sd'] < 200) 
                    & (df['blank_ref_pct'] > 0) 
                    & (df['cs_spike_ratio'] < 5)
                    & (df['host_avg_txn'] > 18) 
                    & (-df['cs_host'] .str.contains(r' [0-9)+. [0-9]+. (0-9)+. (0-9)+'))] 



In [ ]:
x = new df front d.cs host.value_counts() 
x = x[x >c 20] 

new_df_front_d = new df_front_d[-new_df_front_d.cs_host.isin(x.index)] 
new_df_front_d['source'] = 'DomainFronted' 
new_df_front_d["whois_registration_dt'] = '0000/00/00' 
new_df_front_d['whois_registrant'] = 'UNKNOWN' 
new_df_front_d['risk_score'] = 0 
## Write the final output file 
new_df_reg.to_csv(out_reg, header-True, index-False) 

In [ ]:
new_df_front_d.to_csv(out_df, header-True, index-Falae) 
print('End Time:', datetime.now()) 